In [6]:
import pandas as pd
import mysql.connector
import json

from google.cloud import secretmanager

PROJECT_ID = 'alpha-274108'

def access_secret_version(self,project_id, secret_id, version_id):
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """
    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = client.secret_version_path(project_id, secret_id, version_id)

    # Access the secret version.
    response = client.access_secret_version(name)
    
    payload = response.payload.data.decode('UTF-8')
    return format(payload)


HOSTNAME = access_secret_version(PROJECT_ID, 'CHOST-A', 'latest'),
USER = access_secret_version(PROJECT_ID, 'CHOST-A-USER', 'latest'),
PASSWORD = access_secret_version(PROJECT_ID, 'CHOST-A-PASS', 'latest'),
DATABASE = 'sydney_zomato_database'


In [2]:
def getConnection(host_url,user_name,db_passwd,db_name):
    mydb = mysql.connector.connect(
                host=HOSTNAME,
                user=USER,
                passwd=PASSWORD,
                database=DATABASE
    )

    mycursor = mydb.cursor()
    
    sql = 'SELECT t.id,ee.id as group_id FROM (SELECT e.id , r.id as rid, r.group_id FROM entity e, restaurant_group r WHERE r.id = e.entity_id) t, entity ee WHERE ee.entity_id = t.group_id'
    
    mycursor.execute(sql)
    df = pd.DataFrame(mycursor.fetchall())
    df.columns = ['id','group_id']
    return df


In [4]:
group_df = getConnection('159.65.149.50','root','krowd_voljin_db','sydney_zomato_database')

In [5]:
group_df.head()

,id,group_id
0,service_nsw_157960380376911u32J00n026T0a22e1,service_nsw_157960380376911u32J00n026T0a22e1
1,service_nsw_157960380384232a122u02T6301J0en0,service_nsw_157960380384232a122u02T6301J0en0
2,service_nsw_15796038039098en021006120Ja2T23u,service_nsw_15796038039098en021006120Ja2T23u
3,service_nsw_15796038039849T210013Ju202n602ea,service_nsw_15796038039849T210013Ju202n602ea
4,service_nsw_157960380405246a0Tne4J100222102u,service_nsw_157960380405246a0Tne4J100222102u


# listGroupDict json

In [10]:
listGroupdf = pd.DataFrame(group_df.groupby('group_id')['id'].apply(list))

In [12]:
listGroupdf.head(10)

,id
group_id,
service_nsw_157960380376911u32J00n026T0a22e1,[service_nsw_157960380376911u32J00n026T0a22e1]
service_nsw_157960380384232a122u02T6301J0en0,[service_nsw_157960380384232a122u02T6301J0en0]
service_nsw_15796038039098en021006120Ja2T23u,[service_nsw_15796038039098en021006120Ja2T23u]
service_nsw_15796038039849T210013Ju202n602ea,[service_nsw_15796038039849T210013Ju202n602ea]
service_nsw_157960380405246a0Tne4J100222102u,[service_nsw_157960380405246a0Tne4J100222102u]
service_nsw_157960380412230402nJa002126u2eT1,[service_nsw_157960380412230402nJa002126u2eT1]
service_nsw_15796038041937nu20210040eaJT1262,"[service_nsw_15796038041937nu20210040eaJT1262,..."
service_nsw_15796038042598uJ4102nT012020ea62,"[service_nsw_15796038042598uJ4102nT012020ea62,..."
service_nsw_1579603804337801u224J000n61a2eT2,[service_nsw_1579603804337801u224J000n61a2eT2]


In [25]:
listGroupDict = dict(zip(listGroupdf.index,listGroupdf.id))
listGroupDict = [listGroupDict]
with open('listGroupDict.json', 'w') as fp:
    json.dump(listGroupDict, fp)

# groupDict json

In [22]:
groupDict = dict(zip(group_df.id,group_df.group_id))
groupDict = [groupDict]
with open('groupDict.json', 'w') as fp:
    json.dump(groupDict, fp)